# Funcionarios Públicos

El presente notobook permite descargar, limpiar y almacenar los datos provenietes del [repositorio de datos abiertos de personal](https://www.portaltransparencia.cl/PortalPdT/web/guest/opendata#_48_INSTANCE_GI66ozEZ7DNy_=dataset%2Ftransparencia-activa-publicada-en-el-portal) del Consejo Para La Transparencia (CPLT).
Para ejecutar este notebook, se tienen los siguiente prerequisitos:

- Tener instalado el gestor de base de datos MySQL en algún servidor accesible por el usuario con permisos de escritura.
- Tener instalado Anaconda Python y creado un ambiente que contenga las siguientes librerías: pandas, numpy, time, date, datetime, functools, urllib, json, mysql.connector, sqlalchemy


### Descarga de datos desde el Portal de Transparencia
⚠️ **La ejecución de este notebook requiere de al menos 32GB de RAM**, de preferencia 64 GB de RAM par evitar uso de SWAP memeroy, especialmente si utilizas discos HDD y no SSD

In [1]:
# librerias necesarias
import pandas as pd

# Descarga de las cuatro bases (planta, contrata, honorarios, cod trabajo)
#Planta
planta = pd.read_csv('http://www.cplt.cl/transparencia_activa/datoabierto/archivos/TA_PersonalPlanta.csv', encoding = 'cp1252', sep = ';')
planta.to_csv('G:/temp/planta.csv')
planta['Regimen'] = 'Planta'
#Contrata
contrata = pd.read_csv('http://www.cplt.cl/transparencia_activa/datoabierto/archivos/TA_PersonalContrata.csv', encoding = 'cp1252', sep = ';')
contrata['Regimen'] = 'Contrata'
#Honorarios
honorarios = pd.read_csv('http://www.cplt.cl/transparencia_activa/datoabierto/archivos/TA_PersonalContratohonorarios.csv', encoding = 'cp1252', sep = ';')
honorarios['Regimen'] = 'Honorarios'
honorarios = honorarios.rename(columns={'remuneracionbruta': 'remuneracionbruta_mensual'})
#Código del Trabajo
codTrabajo = pd.read_csv('http://www.cplt.cl/transparencia_activa/datoabierto/archivos/TA_PersonalCodigotrabajo.csv', encoding = 'cp1252', sep = ';')
codTrabajo['Regimen'] = 'Código del Trabajo'

planta.to_csv('G:/temp/planta.csv')
contrata.to_csv('G:/temp/contrata.csv')
honorarios.to_csv('G:/temp/honorarios.csv')
codTrabajo.to_csv('G:/temp/codTrabajo.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
import pandas as pd

# Descarga de las cuatro bases (planta, contrata, honorarios, cod trabajo)
#Planta
planta = pd.read_csv('G:/temp/planta.csv', sep = ',')
planta['Regimen'] = 'Planta'
#Contrata
contrata = pd.read_csv('G:/temp/contrata.csv', sep = ',')
contrata['Regimen'] = 'Contrata'
#Honorarios
honorarios = pd.read_csv('G:/temp/honorarios.csv', sep = ',')
honorarios['Regimen'] = 'Honorarios'
honorarios = honorarios.rename(columns={'remuneracionbruta': 'remuneracionbruta_mensual'})
#Código del Trabajo
codTrabajo = pd.read_csv('G:/temp/codTrabajo.csv', sep = ',')
codTrabajo['Regimen'] = 'Código del Trabajo'

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (11,27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
#Unimos los datasets
funHist = pd.concat([planta, contrata, codTrabajo, honorarios], sort = True)


In [4]:
funHist.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38013788 entries, 0 to 4875321
Data columns (total 34 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Horas extra diurnas        float64
 1   Horas extra festivas       float64
 2   Horas extra nocturnas      float64
 3   Materno                    object 
 4   Mes                        object 
 5   Nombres                    object 
 6   Pago extra diurnas         float64
 7   Pago extra festivas        float64
 8   Pago extra nocturnas       float64
 9   Paterno                    object 
 10  Regimen                    object 
 11  Tipo Estamento             object 
 12  Tipo Unidad monetaria      object 
 13  Tipo cargo                 object 
 14  Unnamed: 0                 int64  
 15  activado                   object 
 16  anyo                       int64  
 17  asignaciones               object 
 18  camino                     object 
 19  descripcion_funcion        object 
 20  e

#### **Limpieza de cada campo**

In [5]:
# eliminamos campos innecesarios
funHist = funHist.drop(columns = {'camino', 'enlace', 'observaciones', 'remuliquida_mensual'})

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
funHist.iloc[1000000:1000004]

In [6]:
#Limpieza de las horas extras

funHist = funHist.rename(columns={'Pago extra diurnas':'montoHExDiu', 'Horas extra diurnas':'cantHExDiu'
                                , 'Pago extra nocturnas':'montoHExNoc', 'Horas extra nocturnas':'cantHExNoc'
                                , 'Pago extra festivas':'montoHExFes', 'Horas extra festivas':'cantHExFes'})

#Diurnas
funHist['montoHExDiu'] = pd.to_numeric(funHist['montoHExDiu'], errors = 'coerce') # convertimos a números
funHist['montoHExDiu'] = funHist['montoHExDiu'].fillna(0.0) # imputamos missing

funHist['cantHExDiu'] = pd.to_numeric(funHist['cantHExDiu'], errors = 'coerce') # convertimos a números
funHist['cantHExDiu'] = funHist['cantHExDiu'].fillna(0.0) # imputamos missing

#Nocturnas
funHist['montoHExNoc'] = pd.to_numeric(funHist['montoHExNoc'], errors = 'coerce') # convertimos a números
funHist['montoHExNoc'] = funHist['montoHExNoc'].fillna(0.0) # imputamos missing

funHist['cantHExNoc'] = pd.to_numeric(funHist['cantHExNoc'], errors = 'coerce') # convertimos a números
funHist['cantHExNoc'] = funHist['cantHExNoc'].fillna(0.0) # imputamos missing

#Festivas
funHist['montoHExFes'] = pd.to_numeric(funHist['montoHExFes'], errors = 'coerce') # convertimos a números
funHist['montoHExFes'] = funHist['montoHExFes'].fillna(0.0) # imputamos missing

funHist['cantHExFes'] = pd.to_numeric(funHist['cantHExFes'], errors = 'coerce') # convertimos a números
funHist['cantHExFes'] = funHist['cantHExFes'].fillna(0.0) # imputamos missing

In [7]:
#Limpieza de fechas
funHist['fecha_ingreso'] = funHist['fecha_ingreso'].str.replace(' 00:00:00.000', '').str.replace('1-01-01 00:00:00', '')
funHist['fecha_ingreso'] = pd.to_datetime(funHist.fecha_ingreso, errors = 'coerce').dt.strftime('%d/%m/%Y')

funHist['fecha_publicacion'] = funHist['fecha_publicacion'].str.replace(' 00:00:00.000', '').str.replace('1-01-01 00:00:00', '')
funHist['fecha_publicacion'] = pd.to_datetime(funHist.fecha_publicacion, errors = 'coerce').dt.strftime('%d/%m/%Y')

funHist['fecha_termino'] = funHist['fecha_termino'].str.replace(r'Indefinido|Sin fecha término','').str.replace(r'31\/12\/(([2-9][0-9][3-9][0-9])|([3-9][0-9][0-9][0-9])|([2-9][1-9][0-9][0-9]))', '').fillna('')
funHist['fecha_termino'] = pd.to_datetime(funHist['fecha_termino'], errors = 'coerce').dt.strftime('%d/%m/%Y')

C:\Users\Jose\AppData\Local\Temp\2/ipykernel_11788/1475773041.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  funHist['fecha_ingreso'] = funHist['fecha_ingreso'].str.replace(' 00:00:00.000', '').str.replace('1-01-01 00:00:00', '')
C:\Users\Jose\AppData\Local\Temp\2/ipykernel_11788/1475773041.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  funHist['fecha_publicacion'] = funHist['fecha_publicacion'].str.replace(' 00:00:00.000', '').str.replace('1-01-01 00:00:00', '')
C:\Users\Jose\AppData\Local\Temp\2/ipykernel_11788/1475773041.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  funHist['fecha_termino'] = funHist['fecha_termino'].str.replace(r'Indefinido|Sin fecha término','').str.replace(r'31\/12\/(([2-9][0-9][3-9][0-9])|([3-9][0-9][0-9][0-9])|([2-9][1-9][0-9][0-9]))', '').fillna('')


In [8]:
funHist.keys()

Index(['cantHExDiu', 'cantHExFes', 'cantHExNoc', 'Materno', 'Mes', 'Nombres',
       'montoHExDiu', 'montoHExFes', 'montoHExNoc', 'Paterno', 'Regimen',
       'Tipo Estamento', 'Tipo Unidad monetaria', 'Tipo cargo', 'Unnamed: 0',
       'activado', 'anyo', 'asignaciones', 'descripcion_funcion',
       'fecha_ingreso', 'fecha_publicacion', 'fecha_termino', 'grado_eus',
       'organismo_codigo', 'organismo_nombre', 'pago_mensuales', 'region',
       'remuneracionbruta_mensual', 'tipo_calificacionp', 'viaticos'],
      dtype='object')

In [9]:
#limpieza de Tipo Cargo
funHist['Tipo cargo'] = funHist['Tipo cargo'].str.title()

#Limpieza de años
funHist.loc[funHist['anyo'] < 1900, 'anyo'] = 0
funHist.loc[funHist['anyo'] > 2021, 'anyo'] = 0

#Limpieza de descripciones
funHist['descripcion_funcion'] = funHist['descripcion_funcion'].str.title()

#Limpieza de nombres
funHist['nombre'] = funHist['Nombres'].str.title()

#Limpieza de Organismo
funHist['organismo_nombre'] = funHist['organismo_nombre'].str.title()

#Limpieza de región
funHist['region'] = funHist['region'].str.title()

#Limpieza de tipo de calificación
funHist['tipo_calificacionp'] = funHist['tipo_calificacionp'].str.title()   

In [10]:
# Limpiamos las rentas que no están en pesos
funHist.loc[(funHist['Tipo Unidad monetaria']!='Pesos') & (funHist['remuneracionbruta_mensual'] > 30000) , 'Tipo Unidad monetaria'] = 'Pesos'


## ***Pendiente limpieza a los grados en super complejo de abordar

#### Descarga de datos para realizar transformaciones de moneda y correción monentaria

In [11]:
#Imports necesarios
import time #librería necesaria para medir el tiempo de ejecución 
import datetime
import pandas as pd
import numpy as np
from functools import reduce
import urllib.request
import json

# Creamos los dataframes vacios para almacenar los valores de UTM y Fechas
dolarDiarioDF = pd.DataFrame(columns=['Fecha', 'Dolar'])
CLFDiarioDF = pd.DataFrame(columns=['Fecha', 'CLF'])
euroDiarioDF = pd.DataFrame(columns=['Fecha', 'Euro'])
utmDF = pd.DataFrame(columns=['Fecha', 'UTM'])
ipcDF = pd.DataFrame(columns=['Fecha', 'IPC'])

# Generamos el ciclo para recorrer los años y leer desde la API los datos de UTM
for i in range(2008, 2021):
    with urllib.request.urlopen("https://mindicador.cl/api/dolar/"+str(i)) as url:
        dolar = json.loads(url.read().decode())
    
    with urllib.request.urlopen("https://mindicador.cl/api/uf/"+str(i)) as url:
        uf = json.loads(url.read().decode())
    
    with urllib.request.urlopen("https://mindicador.cl/api/euro/"+str(i)) as url:
        euro = json.loads(url.read().decode())
    
    with urllib.request.urlopen("https://mindicador.cl/api/utm/"+str(i)) as url:
        utm = json.loads(url.read().decode())
        
    with urllib.request.urlopen("https://mindicador.cl/api/ipc/"+str(i)) as url:
        ipc = json.loads(url.read().decode())
    
    # Generamos los ciclos en que se añaden los dataframes los valores obtenidos desde la API    
    for x in range(len(dolar['serie'])):
        dolarDiarioDF = dolarDiarioDF.append({'Fecha' : dolar['serie'][x]['fecha'][0:7] , 'Dolar' : dolar['serie'][x]['valor']} , ignore_index=True) 
    
    for x in range(len(uf['serie'])):
        CLFDiarioDF = CLFDiarioDF.append({'Fecha' : uf['serie'][x]['fecha'][0:7] , 'CLF' : uf['serie'][x]['valor']} , ignore_index=True)
    
    for x in range(len(euro['serie'])):
        euroDiarioDF = euroDiarioDF.append({'Fecha' : euro['serie'][x]['fecha'][0:7] , 'Euro' : euro['serie'][x]['valor']} , ignore_index=True)
    
    for x in range(len(utm['serie'])):
        utmDF = utmDF.append({'Fecha' : utm['serie'][x]['fecha'][0:7] , 'UTM' : utm['serie'][x]['valor']} , ignore_index=True)
        
    for x in range(len(ipc['serie'])):
        ipcDF = ipcDF.append({'Fecha' : ipc['serie'][x]['fecha'][0:7] , 'IPC' : ipc['serie'][x]['valor']} , ignore_index=True)


# Agrupamos por promedio para efectos prácticos
dolarDF = dolarDiarioDF.groupby(['Fecha'])[['Dolar']].mean()
CLFDF = CLFDiarioDF.groupby(['Fecha'])[['CLF']].mean()
euroDF = euroDiarioDF.groupby(['Fecha'])[['Euro']].mean()

# Unimos los dataframes y lo guardamos en uno nuevo
monedas = reduce(lambda x,y: pd.merge(x,y, on='Fecha', how='left'), [dolarDF, CLFDF, euroDF, utmDF, ipcDF])

# calculamos el IPC en términos de su valor en índice
monedas['IPC'] = monedas['IPC'].fillna(0)
monedas['ipcIndices'] = 0.0
for index, row in monedas.iterrows():
    if index == 0:
        monedas['ipcIndices'][index] =  72.0007284469403   
    else:
        monedas['ipcIndices'][index] = monedas['ipcIndices'][index - 1] + monedas['IPC'][index] * monedas['ipcIndices'][index - 1] / 100

# en base al valor índice calculamos los factores de corrección y limpiamos el dataframe
monedas['factorIPC'] = 0.0
for index, row in monedas.iterrows():
    monedas['factorIPC'][index] = monedas['ipcIndices'][len(monedas)-1] / monedas['ipcIndices'][index]
    
monedas = monedas.drop(columns = ['IPC'])


C:\Users\Jose\AppData\Local\Temp\2/ipykernel_11788/1340341544.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monedas['ipcIndices'][index] =  72.0007284469403
C:\Users\Jose\AppData\Local\Temp\2/ipykernel_11788/1340341544.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monedas['ipcIndices'][index] = monedas['ipcIndices'][index - 1] + monedas['IPC'][index] * monedas['ipcIndices'][index - 1] / 100
C:\Users\Jose\AppData\Local\Temp\2/ipykernel_11788/1340341544.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

In [12]:
#elimación de datos de IPC innecesarios
monedas = monedas.drop(columns = {'ipcIndices'})

In [13]:
#creación del campo mes como "numero" para hacer merge con monedas
funHist.loc[funHist['Mes']=='Enero', 'mesN'] = '01'
funHist.loc[funHist['Mes']=='Febrero', 'mesN'] = '02'
funHist.loc[funHist['Mes']=='Marzo', 'mesN'] = '03'
funHist.loc[funHist['Mes']=='Abril', 'mesN'] = '04'
funHist.loc[funHist['Mes']=='Mayo', 'mesN'] = '05'
funHist.loc[funHist['Mes']=='Junio', 'mesN'] = '06'
funHist.loc[funHist['Mes']=='Julio', 'mesN'] = '07'
funHist.loc[funHist['Mes']=='Agosto', 'mesN'] = '08'
funHist.loc[funHist['Mes']=='Septiembre', 'mesN'] = '09'
funHist.loc[funHist['Mes']=='Octubre', 'mesN'] = '10'
funHist.loc[funHist['Mes']=='Noviembre', 'mesN'] = '11'
funHist.loc[funHist['Mes']=='Diciembre', 'mesN'] = '12'

In [14]:
#creación del campo año mes para fusionar con monedas
funHist['Fecha'] = funHist['anyo'].astype(str) + '-' + funHist['mesN']

In [15]:
# merge con monedas
funHist = funHist.merge(monedas, how = 'left', left_on = 'Fecha', right_on = 'Fecha')

In [16]:
# Calculo de remuneración bruta mensual en pesos reales
funHist['RemBrutaPesosReal'] = 0
funHist.loc[funHist['Tipo Unidad monetaria']=='Pesos', 'RemBrutaPesosReal'] = funHist['remuneracionbruta_mensual'] * funHist['factorIPC']
funHist.loc[funHist['Tipo Unidad monetaria']=='Dólares', 'RemBrutaPesosReal'] = funHist['remuneracionbruta_mensual'] * funHist['factorIPC'] * funHist['Dolar']
funHist.loc[funHist['Tipo Unidad monetaria']=='UTM', 'RemBrutaPesosReal'] = funHist['remuneracionbruta_mensual'] * funHist['factorIPC'] * funHist['UTM']
funHist.loc[funHist['Tipo Unidad monetaria']=='Miles de pesos', 'RemBrutaPesosReal'] = funHist['remuneracionbruta_mensual'] * funHist['factorIPC'] * 1000
funHist.loc[funHist['Tipo Unidad monetaria']=='Euros', 'RemBrutaPesosReal'] = funHist['remuneracionbruta_mensual'] * funHist['factorIPC'] * funHist['Euro']
funHist.loc[funHist['Tipo Unidad monetaria']=='UF', 'RemBrutaPesosReal'] = funHist['remuneracionbruta_mensual'] * funHist['factorIPC'] * funHist['CLF']


In [17]:
# Calculo de montos de horas extrasmensual en pesos reales
funHist['HEDiuReal'] = funHist['montoHExDiu'] * funHist['factorIPC']
funHist['HENocReal'] = funHist['montoHExNoc'] * funHist['factorIPC']
funHist['HEFesReal'] = funHist['montoHExFes'] * funHist['factorIPC']
funHist['ViaticosReal'] = funHist['viaticos'] * funHist['factorIPC']
funHist['HorasExtrasMontoReal'] = funHist['HEDiuReal'] + funHist['HENocReal'] + funHist['HEFesReal']
funHist['HorasExtrasCantTotal'] = funHist['cantHExDiu'] + funHist['cantHExNoc'] + funHist['cantHExFes']


In [18]:
# Eliminamos las monedas a no ser usadas
funHist = funHist.drop(columns = {'factorIPC', 'Dolar', 'UTM', 'Euro', 'CLF'})

In [19]:
funHist['Regimen'] = funHist['Regimen'].astype('category')
funHist['Tipo Estamento'] = funHist['Tipo Estamento'].astype('category')
funHist['Tipo Unidad monetaria'] = funHist['Tipo Unidad monetaria'].astype('category')
funHist['Tipo cargo'] = funHist['Tipo cargo'].astype('category')
funHist['asignaciones'] = funHist['asignaciones'].astype('category')
funHist['descripcion_funcion'] = funHist['descripcion_funcion'].astype('category')
funHist['fecha_ingreso'] = pd.to_datetime(funHist['fecha_ingreso'])
funHist['fecha_publicacion'] = pd.to_datetime(funHist['fecha_publicacion'])
funHist['fecha_termino'] = pd.to_datetime(funHist['fecha_termino'])
funHist['grado_eus'] = funHist['grado_eus'].astype('category')
funHist['nombre'] = funHist['nombre'].astype('category')
funHist['organismo_codigo'] = funHist['organismo_codigo'].astype('category')
funHist['organismo_nombre'] = funHist['organismo_nombre'].astype('category')
funHist['pago_mensuales'] = funHist['pago_mensuales'].astype('category')
funHist['region'] = funHist['region'].astype('category')
funHist['tipo_calificacionp'] = funHist['tipo_calificacionp'].astype('category')
funHist['mesN'] = funHist['mesN'].astype('category')
funHist['Fecha'] = funHist['Fecha'].astype('category')
funHist['RemBrutaPesosReal'] = funHist['RemBrutaPesosReal'].astype(float)

In [20]:
funHist2 = funHist.copy()

### Respaldo de lo trabajado en bbdd

In [21]:
import sqlalchemy as sa
import vertica_python
from sqlalchemy import event

engine2 = sa.create_engine('vertica+vertica_python://dbadmin:1@192.168.2.2:5433/chc')

@event.listens_for(engine2, "before_cursor_execute")
def receive_before_cursor_execute(
       conn, cursor, statement, params, context, executemany
        ):
            if executemany:
                cursor.fast_executemany = True

### Tabla de Servicios

In [22]:
funHist2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38013788 entries, 0 to 38013787
Data columns (total 40 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   cantHExDiu                 float64       
 1   cantHExFes                 float64       
 2   cantHExNoc                 float64       
 3   Materno                    object        
 4   Mes                        object        
 5   Nombres                    object        
 6   montoHExDiu                float64       
 7   montoHExFes                float64       
 8   montoHExNoc                float64       
 9   Paterno                    object        
 10  Regimen                    category      
 11  Tipo Estamento             category      
 12  Tipo Unidad monetaria      category      
 13  Tipo cargo                 category      
 14  Unnamed: 0                 int64         
 15  activado                   object        
 16  anyo                       int64  

In [23]:
funHist2 = funHist2.rename(columns = {'Tipo Estamento':'TipoEstamento', 'Tipo Unidad monetaria':'TipoUnidadMonetaria', 
                                     'Tipo cargo': 'TipoCargo', 'descripcion_funcion':'DescripcionFuncion',
                                     'fecha_ingreso': 'FechaIngreso', 'fecha_publicacion':'FechaPublicacion',
                                     'fecha_termino':'FechaTermino', 'grado_eus':'GradoEus', 'organismo_codigo':'OrganismoCodigo',
                                     'organismo_nombre':'OrganismoNombre', 'pago_mensuales':'PagoMensuales',
                                     'tipo_calificacionp':'TipoCalificacionp', 'remuneracionbruta_mensual':'remuneracionbrutamensual', 'Unnamed: 0':'indice'})

In [24]:
funHist2.to_sql(name='full', con=engine2, if_exists = 'replace', index=False ,chunksize = 60000, schema = 'funcionarios',
               dtype={'Mes': sa.types.VARCHAR(50),
                      'Regimen': sa.types.VARCHAR(50),
                      'TipoEstamento': sa.types.VARCHAR(255),
                      'TipoUnidadMonetaria': sa.types.VARCHAR(50),
                      'TipoCargo': sa.types.VARCHAR(255),
                      'asignaciones': sa.types.VARCHAR(2500),
                      'DescripcionFuncion': sa.types.VARCHAR(8000),
                      'FechaIngreso': sa.types.VARCHAR(50),
                      'FechaPublicacion': sa.types.VARCHAR(50),
                      'FechaTermino': sa.types.VARCHAR(50),
                      'GradoEus': sa.types.VARCHAR(255),
                      'Nombres': sa.types.VARCHAR(500),
                      'Materno': sa.types.VARCHAR(500),
                      'Paterno': sa.types.VARCHAR(500),
                      'OrganismoCodigo': sa.types.VARCHAR(50),
                      'OrganismoNombre': sa.types.VARCHAR(255),
                      'PagoMensuales': sa.types.VARCHAR(255),
                      'region': sa.types.VARCHAR(255),
                      'TipoCalificacionp': sa.types.VARCHAR(2500),
                      'Fecha': sa.types.VARCHAR(255),
                      'mesN': sa.types.VARCHAR(50),
                      'nombre': sa.types.VARCHAR(500),
                      'activado': sa.types.VARCHAR(50)})

In [25]:
Servicios = pd.read_csv('http://www.cplt.cl/transparencia_activa/datoabierto/archivos/Organismos_360.csv', encoding = 'cp1252', sep = ';')
Servicios.to_sql(name='Servicios', con=engine2, if_exists = 'replace', index=False ,chunksize = 1000, schema = 'funcionarios',
               dtype={"Codigo_org": sa.types.VARCHAR(50), 
                        "Organismo" : sa.types.VARCHAR(255), 
                        "Codigo_padre" : sa.types.VARCHAR(255), 
                        "Padre_org" : sa.types.VARCHAR(255), 
                        "Region" : sa.types.VARCHAR(255), 
                        "Municipalidad" : sa.types.VARCHAR(255), 
                        "direccion" : sa.types.VARCHAR(255), 
                        "telefono" : sa.types.VARCHAR(255), 
                        "url_organismo" : sa.types.VARCHAR(255), 
                        "url_orgtransparencia" : sa.types.VARCHAR(255), 
                        "horario_publico" : sa.types.VARCHAR(500), 
                        "nombre_encargado" : sa.types.VARCHAR(255), 
                        "email" : sa.types.VARCHAR(255), 
                        "rut" : sa.types.VARCHAR(255), 
                        "tipo_cuenta" : sa.types.VARCHAR(255), 
                        "banco" : sa.types.VARCHAR(255), 
                        "url_sai" : sa.types.VARCHAR(255), 
                        "fax" : sa.types.VARCHAR(255), 
                        "Ingresa" : sa.types.VARCHAR(255), 
                        "obligadorecibir_sai" : sa.types.VARCHAR(255), 
                        "organismo_autonomo" : sa.types.VARCHAR(255), 
                        "interopera" : sa.types.VARCHAR(255), 
                        "tiene_ta" : sa.types.VARCHAR(255), 
                        "fecha_ta" : sa.types.VARCHAR(255),
                        'activado': sa.types.VARCHAR(50)})